# Export csv file(behavior_ms) from txt file

## Import packages

In [8]:
import csv
import sys
import os
import re
import pandas as pd
minian_path = "."
sys.path.insert(0, minian_path)
from minian.utilities import (
    match_information,
    match_path
)

## File path

In [9]:
dpath="/N/project/Cortical_Calcium_Image/Miniscope data/03.2023_Eighth_group/AA050_D8/2023_03_14/10_33_11/Miniscope_2"
inputfilename = "2023-03-14_10h27m_Subject YC_AA050.txt"
mouseID, day, session = match_information(dpath)
mouse_path, video_path = match_path(dpath)
outputfilename = "behavior_ms.csv"
timestamp_path=os.path.join(video_path, "timeStamps.csv")

## reformat

In [10]:
timestamp_pd=pd.read_csv(timestamp_path)

In [11]:
def reformat(data_array,col_name, time_pd):
    output_pd = time_pd
    output_pd.insert(len(output_pd.columns), col_name, 0)
    i = 0
#     timestamp= data_array[i] * 1000
#     print(output_pd.loc[(output_pd['Time Stamp (ms)'].shift(0)<= timestamp) & (output_pd['Time Stamp (ms)'].shift(-1) > timestamp)])
    while (i < len(data_array)):
        timestamp = data_array[i] * 1000
        output_pd.loc[(output_pd['Time Stamp (ms)'].shift(0)<= timestamp) & (output_pd['Time Stamp (ms)'].shift(-1)>timestamp), col_name] += 1
        i += 1
#         print(output_pd.loc[(output_pd['Time Stamp (ms)'].shift(0)<= timestamp) & (output_pd['Time Stamp (ms)'].shift(-1) > timestamp)])
    print("{}: {}".format(col_name,output_pd[col_name].sum()))
    return output_pd

In [12]:
alp_number: int
ialp_number: int
rnfs_number: int
X_array = []
Y_array = []
Z_array = []
file_path = os.path.join(mouse_path, inputfilename)
behavior_timeStamp = list(csv.reader(open(file_path)))
# for line in behavior_timeStamp:
#     print(line)
alp_number = int(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[13][0])[0]))
ialp_number = int(
    float(re.findall(r"\d+\.?\d*", behavior_timeStamp[21][0])[0]))
rnfs_number = int(
    float(re.findall(r"\d+\.?\d*", behavior_timeStamp[14][0])[0]))
# alp=re.findall(r"\d+\.?\d*", )
print("mouseID: {}, day: {}, session: {}".format(mouseID, day, session))
print("alp_number: {}".format(alp_number))
print("ialp_number: {}".format(ialp_number))
print("rnfs_number: {}".format(rnfs_number))

print(behavior_timeStamp[36][0])  # X: reinforcement
for i in range(37, 237):
    for j in range(1, 6):
        X_array.append(
            float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
# print(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[237][0])[1]))
X_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[237][0])[1]))
X_array = list(filter(lambda x: x != 0, X_array))

print(behavior_timeStamp[238][0])  # Y: active lever press
for i in range(239, 439):
    for j in range(1, 6):
        Y_array.append(
            float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
Y_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[439][0])[1]))
Y_array = list(filter(lambda x: x != 0, Y_array))

print(behavior_timeStamp[440][0])  # Z: inactive lever press
for i in range(441, 641):
    for j in range(1, 6):
        Z_array.append(
            float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
Z_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[641][0])[1]))
Z_array = list(filter(lambda x: x != 0, Z_array))

res = reformat(X_array, "RNFS", timestamp_pd)
# print(res)
res = reformat(Y_array, "ALP", res)
# print(res)
res = reformat(Z_array, "IALP", res)
# print(res)

res.to_csv(os.path.join(mouse_path, mouseID+"_" +
           day+"_"+"behavior_ms.csv"), index=None)

mouseID: AA050, day: D8, session: None
alp_number: 38
ialp_number: 12
rnfs_number: 26
X:
Y:
Z:
RNFS: 26
ALP: 38
IALP: 12


# Check

In [13]:
check_pd = pd.read_csv(os.path.join(mouse_path,mouseID+"_"+day+"_"+"behavior_ms.csv"))

In [14]:
print(check_pd.loc[(check_pd["RNFS"]>1) | (check_pd["ALP"]>1) | (check_pd["IALP"]>1)])

Empty DataFrame
Columns: [Frame Number, Time Stamp (ms), Buffer Index, RNFS, ALP, IALP]
Index: []
